<a href="https://colab.research.google.com/github/Gratisfo/Parentents-and-children/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

inspires by this article https://towardsdatascience.com/unsupervised-sentiment-analysis-a38bf1906483

In [98]:
import pandas as pd
data = pd.read_csv('data.csv')
del data['Unnamed: 0']
data = data.dropna().reset_index(drop=True)
data

,speaker,text,role,date,title
0,#taratora,"И ты еще, мошенник, смеешь мне указывать! Да з...",parent,1788,krylov-prokazniki
1,#taratora,То есть чего-нибудь хорошенького.,parent,1788,krylov-prokazniki
2,#taratora,"Как, бездельник! да разве не для того он сюда ...",parent,1788,krylov-prokazniki
3,#taratora,"Да его ль вина, бестия, что вы все здесь ходит...",parent,1788,krylov-prokazniki
4,#taratora,Я окончала!.. Как я рада! Элегия эта беспример...,parent,1788,krylov-prokazniki
...,...,...,...,...,...
17059,#sineus,"Ты, сказывают, хотел Славян и прочие народы от...",children,1786,ekaterina-vtoraja-iz-zhizni-rjurika
17060,#truvor,Какие же ты имел при том намерения?,children,1786,ekaterina-vtoraja-iz-zhizni-rjurika
17061,#oskold,"К чему прение тут, где дело само по себе ясно?",children,1786,ekaterina-vtoraja-iz-zhizni-rjurika
17062,#rjurik,"Бодрость духа твоего, князь Вадим, не унывает;...",parent,1786,ekaterina-vtoraja-iz-zhizni-rjurika


In [ ]:
!pip install razdel

In [ ]:
!pip install pymorphy2[fast]

In [6]:
import re
import pymorphy2
import nltk
from razdel import sentenize, tokenize
import string

In [7]:
def preproc_text(text):
  # tokenize
  tokens = [_.text for _ in list(tokenize(text))]
  
  # lower register
  tokens = [w.lower() for w in tokens]
  
  # remove puntuation
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  
  # remove numbers, save only letters
  words = [word for word in stripped if word.isalpha()]
  
  return words

In [99]:
data['clean_text'] = data['text'].apply(lambda x: preproc_text(x))

# retaining all rows with sentences with a length of at least 3 words.
data = data[data['clean_text'].map(len) > 2]

In [100]:
file_model = data.copy()
file_model = file_model[file_model.clean_text.str.len()>1]

In [102]:
del file_model['speaker'] 
del file_model['text'] 
del file_model['role'] 
del file_model['date'] 
del file_model['title'] 

In [105]:
file_model = file_model.rename(columns={'clean_text':'text_list'}).reset_index(drop=True)

In [ ]:
sent = [row for row in file_model.text_list]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

In [16]:
import multiprocessing

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

In [ ]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.save("word2vec.model")

In [127]:
final_file = file_model.copy()
final_file['text'] = final_file.text_list
final_file.text = final_file.text.str.join(' ')
final_file['text_bi'] = final_file.text_list.apply(lambda x: ' '.join(bigram[x]))

In [128]:
final_file

,text_list,text,text_bi
0,"[и, ты, еще, мошенник, смеешь, мне, указывать,...",и ты еще мошенник смеешь мне указывать да знае...,и ты еще мошенник смеешь мне указывать да знае...
1,"[то, есть, чегонибудь, хорошенького]",то есть чегонибудь хорошенького,то_есть чегонибудь хорошенького
2,"[как, бездельник, да, разве, не, для, того, он...",как бездельник да разве не для того он сюда вх...,как бездельник да разве не для_того он сюда вх...
3,"[да, его, ль, вина, бестия, что, вы, все, здес...",да его ль вина бестия что вы все здесь ходите ...,да его ль вина бестия что вы все здесь ходите ...
4,"[я, окончала, как, я, рада, элегия, эта, беспр...",я окончала как я рада элегия эта беспримерна а...,я окончала как я рада элегия эта беспримерна а...
...,...,...,...
14883,"[ты, сказывают, хотел, славян, и, прочие, наро...",ты сказывают хотел славян и прочие народы отвл...,ты сказывают хотел славян и прочие народы отвл...
14884,"[какие, же, ты, имел, при, том, намерения]",какие же ты имел при том намерения,какие же ты имел при_том намерения
14885,"[к, чему, прение, тут, где, дело, само, по, се...",к чему прение тут где дело само по себе ясно,к_чему прение тут где дело само_по себе ясно
14886,"[бодрость, духа, твоего, князь, вадим, не, уны...",бодрость духа твоего князь вадим не унывает он...,бодрость_духа твоего_князь вадим не унывает он...


In [114]:
word_vectors = Word2Vec.load("word2vec.model").wv

INFO - 17:38:28: loading Word2Vec object from word2vec.model
INFO - 17:38:28: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 17:38:28: setting ignored attribute vectors_norm to None
INFO - 17:38:28: loading vocabulary recursively from word2vec.model.vocabulary.* with mmap=None
INFO - 17:38:28: loading trainables recursively from word2vec.model.trainables.* with mmap=None
INFO - 17:38:28: setting ignored attribute cum_table to None
INFO - 17:38:28: loaded word2vec.model


In [22]:
from sklearn.cluster import KMeans

In [115]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))

In [116]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

INFO - 17:38:55: precomputing L2-norms of word weight vectors


[('своею', 0.9999736547470093),
 ('чужих_руках', 0.9999713897705078),
 ('никаких', 0.9999709129333496),
 ('глаза', 0.9999700784683228),
 ('надо_было', 0.9999697208404541),
 ('квартиру', 0.999969482421875),
 ('с_отцом', 0.9999693632125854),
 ('ято', 0.9999693632125854),
 ('этим', 0.9999693036079407),
 ('добрым', 0.9999691247940063)]

In [117]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [118]:
import numpy as np
words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])

In [78]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [79]:
words.head(10)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,и,"[-0.08439547, 0.03545999, 0.0011736394, -0.024...",1,1,104.930127,104.930127
1,ты,"[-0.085728966, 0.03549873, 0.0010065425, -0.02...",1,1,107.373913,107.373913
2,еще,"[-0.08435217, 0.034467123, 0.00089435134, -0.0...",1,1,114.833415,114.833415
3,мошенник,"[-0.08540243, 0.035111163, 0.00021683254, -0.0...",1,1,102.986599,102.986599
4,смеешь,"[-0.08515904, 0.034387264, 0.0013560787, -0.02...",1,1,111.957265,111.957265
5,мне,"[-0.08463823, 0.03444994, 0.00067324465, -0.02...",1,1,109.610189,109.610189
6,да,"[-0.085733086, 0.03511489, 0.00018543078, -0.0...",1,1,105.372887,105.372887
7,знаешь_ли,"[-0.08511629, 0.0339389, 0.00036508343, -0.024...",1,1,118.670505,118.670505
8,что,"[-0.08511542, 0.03439768, 0.00023882299, -0.02...",0,-1,105.699109,-105.699109
9,я,"[-0.084758684, 0.035062984, 0.0007075275, -0.0...",1,1,98.800546,98.800546


In [80]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from IPython.display import display

In [122]:
sentiment_map = pd.read_csv('sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [124]:
file_weighting

,text_list,text
0,"[и, ты, еще, мошенник, смеешь, мне, указывать,...",и ты еще мошенник смеешь мне указывать да знае...
1,"[то, есть, чегонибудь, хорошенького]",то есть чегонибудь хорошенького
2,"[как, бездельник, да, разве, не, для, того, он...",как бездельник да разве не для того он сюда вх...
3,"[да, его, ль, вина, бестия, что, вы, все, здес...",да его ль вина бестия что вы все здесь ходите ...
4,"[я, окончала, как, я, рада, элегия, эта, беспр...",я окончала как я рада элегия эта беспримерна а...
...,...,...
14883,"[ты, сказывают, хотел, славян, и, прочие, наро...",ты сказывают хотел славян и прочие народы отвл...
14884,"[какие, же, ты, имел, при, том, намерения]",какие же ты имел при том намерения
14885,"[к, чему, прение, тут, где, дело, само, по, се...",к чему прение тут где дело само по себе ясно
14886,"[бодрость, духа, твоего, князь, вадим, не, уны...",бодрость духа твоего князь вадим не унывает он...


In [129]:
file_weighting = final_file.copy()

In [130]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(file_weighting.text_bi)
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(file_weighting.text_bi)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [134]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)  
    return list(map(lambda y:dictionary[f'{y}'], x.text_bi.split()))

In [135]:
replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)#this step takes around 3-4 minutes minutes to calculate

In [136]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [137]:
replaced_closeness_scores = file_weighting.text_bi.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

In [144]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.text_bi]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
replacement_df

,sentiment_coeff,tfidf_scores,sentence,sentiment_rate,prediction
0,"[104.93012701207613, 107.37391317356463, 114.8...","[2.547837917716911, 5.7546496404968455, 4.3449...",и ты еще мошенник смеешь мне указывать да знае...,4238.441608,1
1,"[-118.38301328192786, 118.6938030104928, 0]","[6.970791804529164, 7.90032776315334, 9.915230...",то_есть чегонибудь хорошенького,112.496608,1
2,"[106.39785302470594, -112.09033058321751, 105....","[3.224388506277042, 8.043428606794013, 3.05038...",как бездельник да разве не для_того он сюда вх...,4476.113469,1
3,"[105.37288671469678, -116.25481289667304, 115....","[3.0503830057247447, 4.178658486216413, 6.1540...",да его ль вина бестия что вы все здесь ходите ...,2406.694493,1
4,"[98.80054649396841, 0, 106.39785302470594, 98....","[4.669062062942085, 9.915230783695606, 3.22438...",я окончала как я рада элегия эта беспримерна а...,2459.471943,1
...,...,...,...,...,...
14883,"[107.37391317356463, 106.0646671703799, 114.51...","[2.8773248202484227, 8.528936422575715, 6.5140...",ты сказывают хотел славян и прочие народы отвл...,3711.014760,1
14884,"[-115.03061095297777, -107.84247409154423, 107...","[6.044029772787714, 3.739363513589844, 2.87732...",какие же ты имел при_том намерения,-2598.738160,0
14885,"[-118.52083611659828, 0, 114.3917382177896, -1...","[6.894805897551243, 9.915230783695606, 5.12773...",к_чему прение тут где дело само_по себе ясно,-748.904986,0
14886,"[0, 0, 0, -106.42144309214964, 0, 106.56049622...","[9.50976567558744, 9.50976567558744, 9.9152307...",бодрость_духа твоего_князь вадим не унывает он...,3374.781878,1
